# Projeto Webscraping: Cálculo de Rentabilidade Diária e Anual de Fundos de Ações

**Autor:** Eduardo B. Scheffer

**Data:** 31/08/2020

**Objetivo:** Obter rentabilidade diária e YTD de uma lista de fundos de ações:

* Constellation Ações FIC FIA
* Sqadra Long-Only FIC FIA
* Bogari Value FIC FIA
* Núcleo FIC FIA
* Dynamo Cougar FIA

**Observação:** Abrir o arquivo "Projeto Brasil Capital 2.xlsx" antes de e após rodar este programa, para verificar sua funcionalidade

---
## Método II:
    Extração dos dados das cotas diretamente do site das gestoras
    
   #### Obs.:

    - Constellation não disponibiliza informações das cotas dos fundos ou rentabilidade em seu website.
    
    - Dynamo não possui tabelas em seu website. As ferramentas que eu tinha à disposição não me permitiram extrair essas informações dentro de um tempo razoável 
    
    - Sqadra, Bogari e Nucleo não disponibilizam apenas rentabilidade diária
    
   #### Portanto, serão obtidas as rentabilidades acumuladas do Sqadra Long-Only FIC FIA, Bogari Value FIC FIA e Núcleo FIC FIAe a rentabilidade do dia do Dynamo Cougar FIA através deste método

### Importar bibliotecas:

In [1]:
import pandas as pd
from pandas import ExcelWriter
from datetime import date
from openpyxl import load_workbook

### Definição dos url das gestoras.



In [2]:
asset_url = ["https://constellation.com.br/pra-voce/acoes/", "http://www.squadrainvestimentos.com/fundo-longonly.aspx", "http://bogaricapital.com.br/fundos/fundo-modelo/",
               "https://www.nucleocapital.com.br/nucleo-acoes-fic-fia.php", "https://www.dynamo.com.br/en"]

### Declaração CNPJ dos fundos:

In [3]:
fundos2 = {'Nome Fundo': ['Constellation Ações FIC FIA', 'Sqadra Long-Only FIC FIA', 'Bogari Value FIC FIA', 'Núcleo FIC FIA', 'Dynamo Cougar FIA']}
fundos2 = pd.DataFrame(fundos2, columns = ['Nome Fundo'])
fundos2['Rentabilidade diária'] = ""
fundos2['Rentabilidade YTD'] = ""

### Obtenção das rentabilidades:

In [4]:
for x in range(0, len(fundos2)):
    
    # Cada asset é um caso diferente, pois cada website é diferente em termos de tabelas 
    # Sqadra
    if x == 1:  
        tables = pd.read_html(asset_url[x]) # cria um data frame com as tabelas presentes no site 
        df = tables[0]  # Utiliza-se a primeira tabela
        df =  df.iloc[1:] # Limpa a primeira linha
        df = df[df[0].notna()] # Deleta as linhas vazias
        df.reset_index(drop=True,inplace=True) # Renumera os indices, para começar do 0
        for i in range(len(df)) :
            if(df.at[i,0]) == '2020': 
                fundos2['Rentabilidade YTD'].iloc[x] = float(str(df.at[i,13])) / 10000 # Obtém-se a rentabilidade acumulada de 2020
                break
                
    # Bogari
    if x == 2: 
        tables = pd.read_html(asset_url[x])
        df = tables[0]  # tiliza-se a primeira tabela
        for i in range(len(df)) :
            if(df[df.index == i].iloc[i,0]) == '2020': 
                rentab = df[df.index == i].iloc[i,13]
                rentab = rentab.replace(",",".") ## agora, transformando string para float e então, para porcentagem
                rentab = rentab.replace("%","")
                fundos2['Rentabilidade YTD'].iloc[x] = float(rentab) / 100
                break      
           
    # Núcleo
    if x == 3: 
        tables = pd.read_html(asset_url[x]) # cria um data frame com as tabelas presentes no site 
        df = tables[0]  # Utiliza-se a primeira tabela
        for i in range(len(df)) :
            if(df[df.index == i].iloc[0,0]) == 2020: 
                rentab = df[df.index == i].iloc[0,13]
                rentab = rentab.replace(",",".") ## agora, transformando string para float e então, para porcentagem
                rentab = rentab.replace("%","")
                fundos2['Rentabilidade YTD'].iloc[3] = float(rentab) / 100
                break

### Print dos resultados:

In [5]:
fundos2

,Nome Fundo,Rentabilidade diária,Rentabilidade YTD
0,Constellation Ações FIC FIA,,
1,Sqadra Long-Only FIC FIA,,-0.0244
2,Bogari Value FIC FIA,,0.004
3,Núcleo FIC FIA,,0.0651
4,Dynamo Cougar FIA,,


### Exportar para o Excel em nova aba ("Input")

In [6]:
wb = load_workbook("Projeto Brasil Capital 2.xlsx") #Abre a planilha a ser preenchida

writer = pd.ExcelWriter("Projeto Brasil Capital 2.xlsx")
writer.book = wb
writer.sheets = dict((ws.title, ws) for ws in wb.worksheets) # Possibilita escrever em aba já existente em caso de atualização dos dados

fundos2.to_excel(writer, sheet_name='Input', index = False) # Copia informações do fundo na aba "Input"
writer.save()

### Exportar para o Excel em na aba "Projeto_2020"

In [8]:
ws = wb['Projeto_2020']
for x in range(0, len(fundos2)):
    ws.cell(row = 12+x, column=4).value = fundos2['Rentabilidade diária'].iloc[x]
    ws.cell(row = 12+x, column=5).value = fundos2['Rentabilidade YTD'].iloc[x]
writer.save()